In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
df_ratings = pd.read_csv("ratings.csv")
df_movies = pd.read_csv("movies.csv")

In [6]:
df_ratings = df_ratings.merge(df_movies[['movieId', 'title']],how='left', on='movieId')

In [7]:
df_ratings.groupby('rating').count()

,userId,movieId,timestamp,title
rating,,,,
0.5,1370,1370,1370,1370
1.0,2811,2811,2811,2811
1.5,1791,1791,1791,1791
2.0,7551,7551,7551,7551
2.5,5550,5550,5550,5550
3.0,20047,20047,20047,20047
3.5,13136,13136,13136,13136
4.0,26818,26818,26818,26818
4.5,8551,8551,8551,8551


In [8]:
df_ratings.groupby(by = "userId")["rating"].count().sort_values(ascending = False)

userId
414    2698
599    2478
474    2108
448    1864
274    1346
       ... 
442      20
569      20
320      20
576      20
53       20
Name: rating, Length: 610, dtype: int64

In [9]:
df_ratings.groupby(by = "title")["rating"].count().sort_values(ascending = False)

title
Forrest Gump (1994)                          329
Shawshank Redemption, The (1994)             317
Pulp Fiction (1994)                          307
Silence of the Lambs, The (1991)             279
Matrix, The (1999)                           278
                                            ... 
King Solomon's Mines (1950)                    1
King Solomon's Mines (1937)                    1
King Ralph (1991)                              1
King Kong Lives (1986)                         1
À nous la liberté (Freedom for Us) (1931)      1
Name: rating, Length: 9719, dtype: int64

In [10]:
df_user_item = df_ratings.pivot_table(index=['userId'], columns=['title'], values='rating')

In [11]:
corr_matrix = df_user_item.corr(method='pearson', min_periods=100)

In [12]:
corr_matrix

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Hellboy': The Seeds of Creation (2004),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Round Midnight (1986),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Salem's Lot (2004),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Til There Was You (1997),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
eXistenZ (1999),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
xXx (2002),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
xXx: State of the Union (2005),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
user_ratings = df_user_item.loc[7].dropna()
user_ratings

title
2001: A Space Odyssey (1968)           4.0
A.I. Artificial Intelligence (2001)    4.5
Aladdin (1992)                         3.0
American Beauty (1999)                 4.0
Apocalypse Now (1979)                  4.0
                                      ... 
What Women Want (2000)                 4.0
Wild Wild West (1999)                  1.5
X-Men (2000)                           3.5
X-Men: The Last Stand (2006)           4.0
X2: X-Men United (2003)                4.0
Name: 7, Length: 152, dtype: float64

In [21]:
simCandidates = pd.Series()
for i in range(0, len(user_ratings.index)): # Идем по списку всех фильмов оцененных пользвателем
    sims = corr_matrix[user_ratings.index[i]].dropna() # Извлекаем фильмы, похожие на оцененные данным юзером (1)
    sims = sims.map(lambda x: x * user_ratings[i]) # Умножаем корреляцию на оценку пользователя по фильму
    simCandidates = simCandidates._append(sims) # Добавляем индекс в список сравниваемых кандидатов
simCandidates.sort_values(inplace = True, ascending = False)

C:\Users\Зотов Михаил\AppData\Local\Temp\ipykernel_9824\315929848.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sims = sims.map(lambda x: x * user_ratings[i]) # Умножаем корреляцию на оценку пользователя по фильму
C:\Users\Зотов Михаил\AppData\Local\Temp\ipykernel_9824\315929848.py:5: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  simCandidates = simCandidates._append(sims) # Добавляем индекс в список сравниваемых кандидатов
C:\Users\Зотов Михаил\AppData\Local\Temp\ipykernel_9824\315929848.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a futur

In [22]:
user_ratings.index[5]

'Apollo 13 (1995)'

In [23]:
corr_matrix[user_ratings.index[5]].dropna()

title
Ace Ventura: Pet Detective (1994)       0.096976
Aladdin (1992)                          0.321222
Apollo 13 (1995)                        1.000000
Batman (1989)                           0.075678
Braveheart (1995)                       0.232201
Dances with Wolves (1990)               0.252496
Forrest Gump (1994)                     0.186355
Fugitive, The (1993)                    0.355056
Independence Day (a.k.a. ID4) (1996)    0.302946
Jurassic Park (1993)                    0.309437
Lion King, The (1994)                   0.316054
Pulp Fiction (1994)                    -0.045333
Schindler's List (1993)                 0.330132
Seven (a.k.a. Se7en) (1995)             0.046390
Shawshank Redemption, The (1994)        0.102889
Silence of the Lambs, The (1991)        0.164643
Speed (1994)                            0.311971
Terminator 2: Judgment Day (1991)       0.399526
Toy Story (1995)                        0.191449
True Lies (1994)                        0.253759
Name: Apollo 1

In [24]:
user_ratings[5]

C:\Users\Зотов Михаил\AppData\Local\Temp\ipykernel_9824\2761511157.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  user_ratings[5]


4.5

In [25]:
sims = corr_matrix[user_ratings.index[5]].dropna()

In [26]:
sims = sims.map(lambda x: x * user_ratings[5])

C:\Users\Зотов Михаил\AppData\Local\Temp\ipykernel_9824\3692227409.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sims = sims.map(lambda x: x * user_ratings[5])


In [27]:
sims

title
Ace Ventura: Pet Detective (1994)       0.436391
Aladdin (1992)                          1.445498
Apollo 13 (1995)                        4.500000
Batman (1989)                           0.340552
Braveheart (1995)                       1.044904
Dances with Wolves (1990)               1.136230
Forrest Gump (1994)                     0.838598
Fugitive, The (1993)                    1.597750
Independence Day (a.k.a. ID4) (1996)    1.363258
Jurassic Park (1993)                    1.392467
Lion King, The (1994)                   1.422241
Pulp Fiction (1994)                    -0.203999
Schindler's List (1993)                 1.485596
Seven (a.k.a. Se7en) (1995)             0.208755
Shawshank Redemption, The (1994)        0.463001
Silence of the Lambs, The (1991)        0.740894
Speed (1994)                            1.403870
Terminator 2: Judgment Day (1991)       1.797867
Toy Story (1995)                        0.861521
True Lies (1994)                        1.141915
Name: Apollo 1

In [28]:
simCandidates

Silence of the Lambs, The (1991)             5.000000
Back to the Future (1985)                    5.000000
Terminator, The (1984)                       5.000000
Forrest Gump (1994)                          5.000000
Jurassic Park (1993)                         5.000000
                                               ...   
Pulp Fiction (1994)                         -0.376668
Pulp Fiction (1994)                         -0.431635
Twelve Monkeys (a.k.a. 12 Monkeys) (1995)   -0.474771
Pulp Fiction (1994)                         -0.719040
Fargo (1996)                                -0.801146
Length: 697, dtype: float64

In [29]:
simCandidates = simCandidates.groupby(simCandidates.index).sum()
simCandidates.sort_values(inplace = True, ascending = False)

In [30]:
intersection_set = set.intersection(set(simCandidates.index), set(user_ratings.index))
not_watched = list(set(simCandidates.index) - set(user_ratings.index))

In [31]:
print("Intersection: ", len(intersection_set))
print("Not watched: ", len(not_watched))

Intersection:  47
Not watched:  44


In [32]:
filteredSims = simCandidates[not_watched].sort_values(ascending = False)

In [33]:
filteredSims.head(3)

Matrix, The (1999)                                                                30.707095
Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)    22.002139
Shawshank Redemption, The (1994)                                                  18.925514
dtype: float64